In [2]:
# !pip install -q torch
# !pip install -q transformers
# !pip install -q datasets
# !pip install -q peft
# !pip install -U -q sentencepiece

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import os
import torch
import getpass

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "ogbrandt/mistral-7b-non-instruction-pjf-ft"

model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading adapter weights from ogbrandt/mistral-7b-non-instruction-pjf-ft led to unexpected keys not found in the model:  ['model.layers.0.mlp.up_proj.lora_A.default.weight', 'model.layers.0.mlp.up_proj.lora_B.default.weight', 'model.layers.0.mlp.down_proj.lora_A.default.weight', 'model.layers.0.mlp.down_proj.lora_B.default.weight', 'model.layers.1.mlp.up_proj.lora_A.default.weight', 'model.layers.1.mlp.up_proj.lora_B.default.weight', 'model.layers.1.mlp.down_proj.lora_A.default.weight', 'model.layers.1.mlp.down_proj.lora_B.default.weight', 'model.layers.2.mlp.up_proj.lora_A.default.weight', 'model.layers.2.mlp.up_proj.lora_B.default.weight', 'model.layers.2.mlp.down_proj.lora_A.default.weight', 'model.layers.2.mlp.down_proj.lora_B.default.weight', 'model.layers.3.mlp.up_proj.lora_A.default.weight', 'model.layers.3.mlp.up_proj.lora_B.default.weight', 'model.layers.3.mlp.down_proj.lora_A.default.weight', 'model.layers.3.mlp.down_proj.lora_B.default.weight', 'model.layers.4.mlp.up_proj.lor

In [52]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [43]:
text = "How to increase my vertical jump?"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt", add_special_tokens=True).to(device)
outputs = model.generate(**inputs, max_new_tokens=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> How to increase my vertical jump?
What is the best way to increase my vertical jump?

To increase your vertical jump, you need to focus on improving your overall athleticism. This includes working on your agility, speed, and explosiveness. By doing so, you'll not only improve your vertical jump but also enhance your overall athletic performance.

One way to improve your vertical jump is to incorporate agility drills into your training. These drills can help you develop the agility and speed necessary for a higher vertical jump. Additionally, you can incorporate plyometrics into your training, which involves jumping and landing exercises that can improve


In [54]:
import torch
from transformers import BitsAndBytesConfig

# use bfloat for max
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [55]:
model_cd_bf16 = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading adapter weights from ogbrandt/mistral-7b-non-instruction-pjf-ft led to unexpected keys not found in the model:  ['model.layers.0.mlp.up_proj.lora_A.default.weight', 'model.layers.0.mlp.up_proj.lora_B.default.weight', 'model.layers.0.mlp.down_proj.lora_A.default.weight', 'model.layers.0.mlp.down_proj.lora_B.default.weight', 'model.layers.1.mlp.up_proj.lora_A.default.weight', 'model.layers.1.mlp.up_proj.lora_B.default.weight', 'model.layers.1.mlp.down_proj.lora_A.default.weight', 'model.layers.1.mlp.down_proj.lora_B.default.weight', 'model.layers.2.mlp.up_proj.lora_A.default.weight', 'model.layers.2.mlp.up_proj.lora_B.default.weight', 'model.layers.2.mlp.down_proj.lora_A.default.weight', 'model.layers.2.mlp.down_proj.lora_B.default.weight', 'model.layers.3.mlp.up_proj.lora_A.default.weight', 'model.layers.3.mlp.up_proj.lora_B.default.weight', 'model.layers.3.mlp.down_proj.lora_A.default.weight', 'model.layers.3.mlp.down_proj.lora_B.default.weight', 'model.layers.4.mlp.up_proj.lor

In [57]:
prompt = "what is the fastest way to improve vert?"
text = f"<INST> {prompt} </INST>\n"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model_cd_bf16.generate(**inputs, min_length=50, max_length=128)
print(tokenizer.decode(outputs[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> <INST> what is the fastest way to improve vert? </INST>

The fastest way to improve vert is to improve your mechanics. This includes improving your mechanics, improving your mechanics, and improving your mechanics. By focusing on improving your mechanics, you can improve your overall performance and increase your vertical jump by 10-15%. This is a significant improvement and can be achieved in a relatively short period of time.

<INST> what is the fastest way to improve vert? </INST>

The fastest way to improve vert is to improve your mechanics. This includes improving your mechanics, improving your mechanics


In [60]:
# def process_string(input_string):
#     # Step 1: Remove the topmost portion
#     topmost_inst = "<s> <INST> what is the fastest way to improve vert? </INST>"
#     if input_string.startswith(topmost_inst):
#         input_string = input_string[len(topmost_inst):].strip()

#     # Step 2: Split and remove after instruction
#     subsequent_inst = "<INST> what is the fastest way to improve vert? </INST>"
#     if subsequent_inst in input_string:
#         input_string = input_string.split(subsequent_inst)[0].strip()

#     # Step 3: Check for complete sentence
#     if not input_string.endswith('.'):
#         last_period_idx = input_string.rfind('.')
#         input_string = input_string[:last_period_idx + 1]

#     return input_string

# # Test the function
# test_string = "<s> <INST> what is the fastest way to improve vert? </INST>\n\nThe fastest way to improve vert is to improve your mechanics. This includes improving your mechanics, improving your mechanics, and improving your mechanics. By focusing on improving your mechanics, you can improve your overall performance and increase your vertical jump by 10-15%. This is a significant improvement and can be achieved in a relatively short period of time. ends here. bla blaaaaaaa  \n\n<INST> what is the fastest way to improve vert? </INST>\n\nThe fastest way to improve vert is to improve your mechanics. This includes improving your mechanics, improving your mechanics"
# processed_string = process_string(test_string)
# print(processed_string)

The fastest way to improve vert is to improve your mechanics. This includes improving your mechanics, improving your mechanics, and improving your mechanics. By focusing on improving your mechanics, you can improve your overall performance and increase your vertical jump by 10-15%. This is a significant improvement and can be achieved in a relatively short period of time. ends here.


In [64]:
import re

def process_string(input_string):
    # Regular expression to match <INST>...</INST> patterns
    first_inst_pattern = r"<s> <INST>.*?</INST>\n\n"
    inst_pattern = r"<INST>.*?</INST>"

    # Step 1: Remove the topmost <INST>...</INST> portion
    input_string = re.sub(first_inst_pattern, '', input_string, count=1).strip()

    # Step 2: Split and remove after any subsequent <INST>...</INST>
    input_string = re.split(inst_pattern, input_string, maxsplit=1)[0].strip()

    # Step 3: Check for complete sentence
    if not input_string.endswith('.'):
        last_period_idx = input_string.rfind('.')
        input_string = input_string[:last_period_idx + 1]

    return input_string

# Test the updated function with the original test string
test_string = "<s> <INST> what is the fastest way to improve vert? </INST>\n\nThe fastest way to improve vert is to improve your mechanics. This includes improving your mechanics, improving your mechanics, and improving your mechanics. By focusing on improving your mechanics, you can improve your overall performance and increase your vertical jump by 10-15%. This is a significant improvement and can be achieved in a relatively short period of time.\n\n<INST> what is the fastest way to improve vert? </INST>\n\nThe fastest way to improve vert is to improve your mechanics. This includes improving your mechanics, improving your mechanics"
processed_string = process_string(test_string)
print(processed_string)


The fastest way to improve vert is to improve your mechanics. This includes improving your mechanics, improving your mechanics, and improving your mechanics. By focusing on improving your mechanics, you can improve your overall performance and increase your vertical jump by 10-15%. This is a significant improvement and can be achieved in a relatively short period of time.


### Load the dataset

In [29]:
samples = 1000

# Load the dataset
orig_dataset = load_dataset('CarperAI/openai_summarize_comparisons', split='train')

# Filter samples where the prompt length is less than or equal to X words
filtered_dataset = orig_dataset.filter(lambda example: len(example['prompt'].split()) <= 450) # By word

assert samples <= len(filtered_dataset)

# Shuffle and select the first X thousand samples
shuffled_dataset = filtered_dataset.shuffle(seed=42).select(range(samples))

# Extract the desired features.  Renaming chose to response to follow the ppo library requirements.
new_dataset_dict = HFDataset.from_dict({"prompt": shuffled_dataset["prompt"]})

assert len(new_dataset_dict['prompt']) == samples

Found cached dataset parquet (C:/Users/juan_/.cache/huggingface/datasets/CarperAI___parquet/CarperAI--openai_summarize_comparisons-79d2c222a15dc8fb/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
Loading cached processed dataset at C:\Users\juan_\.cache\huggingface\datasets\CarperAI___parquet\CarperAI--openai_summarize_comparisons-79d2c222a15dc8fb\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-013e8ffda183dffa.arrow
Loading cached shuffled indices for dataset at C:\Users\juan_\.cache\huggingface\datasets\CarperAI___parquet\CarperAI--openai_summarize_comparisons-79d2c222a15dc8fb\0.0.0\2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec\cache-7d25ec769eb17cb5.arrow


### Tokenize the dataset

In [30]:
def generate_summaries(example, **kwargs) -> str:
    """
    Generate a summary for a given prompt using a trained policy model.

    Args:
    - prompt (str): The input text for which a summary needs to be generated.
    - model: The trained policy model.
    - tokenizer: The tokenizer used for the policy model.
    - generation_kwargs (dict): Arguments used for response generation.

    Returns:
    - str: Generated summary.
    """

    summaries = {}

    promt               = example['prompt']
    tokenizer           = kwargs['tokenizer']
    model               = kwargs['model']
    number_of_summaries = kwargs['number_of_summaries']
    generation_kwargs   = {
            "temperature": kwargs['temperature'],
            "min_length": kwargs['min_length'],
            "top_k": kwargs['top_k'],
            "top_p": kwargs['top_p'],
            "do_sample": kwargs['do_sample'],
            "max_new_tokens": kwargs['max_new_tokens']
    }

    for i in range(number_of_summaries):

        # Tokenize the prompt
        prompt_tensor = tokenizer.encode(prompt, return_tensors='pt').to(device)

        # Ensure it's only one tensor and check its shape
        assert prompt_tensor.dim() == 2, f"Unexpected tensor shape: {prompt_tensor.shape}"

        # Generate the summary
        summary_tensor = model.generate(input_ids=prompt_tensor, **generation_kwargs)
        # Decode and return the two summaries
        summary = tokenizer.decode(summary_tensor[0], skip_special_tokens=True)
        # Append to the output dictionary
        summaries['summary_' + str(i+1)] = summary

    return summaries

In [31]:
fn_kwargs = {
    "temperature": 1.0,
    "min_length": 5,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "max_new_tokens": 400,
    "number_of_summaries": 2,
    "model": sft_model,
    "tokenizer": sft_tokenizer
}

new_dataset_dict = new_dataset_dict.map(generate_summaries, fn_kwargs=fn_kwargs, batched=False)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [32]:
new_dataset_dict[0]

{'prompt': 'SUBREDDIT: r/AskReddit\nTITLE: Today i had a table call me a god-hating queer loving peice of trash, reddit what\'s the worst customer you\'ve dealt with?\nPOST: I was in a section with another waiter who happens to be gay, when i came up to the table i was greeted with: "wait, you ain\'t queer too are ya? That faggy one came by and i told him i need a new waiter" Shocked and apalled i answered as i polite as i could: "No sir, I am not gay but i do find it appalling the amount of hatred you have for someones entire existence, i think you\'re going to need another waiter because i can\'t take care of you" He then proceeded to call me a "queer loving god-hating piece of trash" Thank god he left after my manager talked to him and asked him to treat his employees with more respect or he wouldn\'t be served. On the plus side the table next to him overheard the entire thing and gave me a $20 tip and told me i handled such an awful situation "eloquently"',
 'summary_1': 'TL;DR: Is

In [33]:
hf_token = getpass.getpass("Enter your HUGGINGFACE TOKEN: ")

In [34]:
new_dataset_dict.push_to_hub('JuanKO/T5_summarization_RLAIF', token=hf_token)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Updating downloaded metadata with the new split.
